In [ ]:
import praw
import datetime

from praw.models import MoreComments
from textblob import TextBlob
import pandas as pd
import nltk
import sys
from os import path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from collections import Counter
import re, string



####for kmeans####
%matplotlib inline

from __future__ import print_function

from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn import metrics

from sklearn.cluster import KMeans, MiniBatchKMeans

import logging
from optparse import OptionParser
import sys
from time import time

import numpy as np

####for afn prop####
from nltk.corpus import stopwords
import sklearn.cluster
import distance

#### Introduction
#### Our goal is to acquire available Reddit data from two internal sources (/r/news and /r/worldnews) to see how they compare and conduct analyzations on.


#### We start by pulling the data from the Reddit API:

In [ ]:
client_id = open("client_id").read()
client_secret = open("client_secret").read()
password = open("password").read()
user_agent = open("user_agent").read()
username = open("username").read()

In [ ]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent,
                     password=password,
                     username=username)

nltk.download('all')

In [ ]:
def getNews(subreddit, n):
    results = list()
    dic = dict()
    # commentDic=dict()
    for submission in reddit.subreddit(subreddit).top(limit=n):
        tempList = list()
        tempList.append(submission.fullname)
        tempList.append(re.sub('[%s]' % re.escape(string.punctuation), '', str.lower(submission.title)))
        tempList.append(datetime.datetime.fromtimestamp(submission.created))
        tempList.append(submission.score)
        tempList.append(submission.url)
        # tempList.append(submission.comments.list())
        # commentDicTemp=dict()
        # for comment in submission.comments:
        #     if isinstance(comment, MoreComments):
        #         continue
        #     else:
        #         commentDicTemp[comment.id]=comment.body
        #         commentDic[comment.id]=comment.body
        # tempList.append(commentDicTemp)
        dic[submission.id] = tempList
    results.append(dic)
    # results.append(commentDic)
    return results

In [ ]:
startTime = datetime.datetime.now()

resultsUS = getNews("news", 999)
usNews = resultsUS[0]
# usComments = resultsUS[1]

In [ ]:
resultsWorld = getNews("worldnews", 999)
worldNews = resultsWorld[0]
# worldComments = resultsWorld[1]

In [ ]:
endTime = datetime.datetime.now()

print(startTime)
print(endTime)
# 10 records ~80 seconds
# 100 records ~800 seconds
print((endTime - startTime).seconds)

In [ ]:
# Step 1: Sentiment Analysis & Descriptive Stats

usSent = list()
worldSent = list()

for k in usNews:
    # print(k)
    title = usNews[k][1]
    n = TextBlob(title)
    # print(n)
    # print(n.sentiment.polarity)
    usSent.append(n.sentiment.polarity)

for k in worldNews:
    # print(k)
    title = worldNews[k][1]
    n = TextBlob(title)
    # print(n)
    # print(n.sentiment.polarity)
    worldSent.append(n.sentiment.polarity)

usSentAvg = sum(usSent) / float(len(usSent))
worldSentAvg = sum(worldSent) / float(len(worldSent))

In [ ]:
UsSubScore = list()
for i in usNews:
    n = (usNews[i][3])
    UsSubScore.append(n)

WorldSubScore = list()
for i in worldNews:
    n = (worldNews[i][3])
    WorldSubScore.append(n)

In [ ]:
# print("US")
# print(usSentAvg)
# print(min(usSent))
# print(max(usSent))
#
# print("World")
# print(worldSentAvg)
# print(min(worldSent))
# print(max(worldSent))
#
print(len(UsSubScore))
print(len(WorldSubScore))
print(len(usSent))
print(len(worldSent))

In [ ]:
News = pd.DataFrame(
    {'World News': worldSent,
     'US News': usSent,
     'World Post Score': WorldSubScore,
     'US Post Score': UsSubScore
     })

# Descriptive Statistics
print(News.describe())

In [ ]:
# Step 2: LSA
#here i put in our custom stopwords to not have but it dont work
custom_stopword=["says","said","say"]
stopwords_custom = set(STOPWORDS)
stopwords_custom.add("said")
stopwords_custom.add("says")
stopwords_custom.add("say")

def LSA(title_list):
    words = []
    for i in title_list:
        words += i.split()
        #below code is edited and not working. i added and custom_stopword and the parenthesis after not in
    filtered_words = [word for word in words if word not in stopwords_custom]
    terms = {}
    for j in filtered_words:
        if j not in terms:
            terms[j] = 1
        else:
            terms[j] += 1
    return terms

In [ ]:
usNews_titles = []
for k in usNews:
    usNews_titles.append(usNews[k][1])
usLSA = LSA(usNews_titles)
print(usLSA)

In [ ]:
worldNews_titles = []
for k in worldNews:
    worldNews_titles.append(worldNews[k][1])
worldLSA = LSA(worldNews_titles)
print(worldLSA)

In [ ]:
# Step 3: By Time Analysis
#print(usNews)

In [ ]:
# first do with us, then do with world
before_trump = list()
after_trump = list()
before_trump_world=list()
after_trump_world = list()
# nov 8 2016
winning_date = datetime.datetime.strptime('20161108', "%Y%m%d")
# cutoff at may 20
todays_date = datetime.datetime.strptime('20170520', "%Y%m%d")
# 194 days difference between winning date and may20 = 20160428
start_date = datetime.datetime.strptime('20160428', "%Y%m%d")

print(winning_date)
print(todays_date)
print(start_date)

In [ ]:
for news_key in usNews:
    timeStamp = (usNews[news_key][2])
    if timeStamp >= winning_date and timeStamp <= todays_date:
        # append to after trump
        after_trump.append(usNews[news_key])
    elif timeStamp >= start_date and timeStamp < winning_date:
        # append to before trump
        before_trump.append(usNews[news_key])
        # else do nothing, out of sample range
for news_key in worldNews:
    timeStamp = (worldNews[news_key][2])
    if timeStamp >= winning_date and timeStamp <= todays_date:
        # append to after trump
        after_trump_world.append(worldNews[news_key])
    elif timeStamp >= start_date and timeStamp < winning_date:
        # append to before trump
        before_trump_world.append(worldNews[news_key])
        # else do nothing, out of sample range



print(len(after_trump))
print(len(before_trump))
print(len(after_trump_world))
print(len(before_trump_world))

In [ ]:
# just the titles
after_trump_titles_only = list()
for l in after_trump:
    after_trump_titles_only.append(l[1])
before_trump_titles_only = list()
for l in before_trump:
    before_trump_titles_only.append(l[1])
after_trump_titles_only_world = list()
for l in after_trump_world:
    after_trump_titles_only_world.append(l[1])
before_trump_titles_only_world = list()
for l in before_trump_world:
    before_trump_titles_only_world.append(l[1])


In [ ]:
ATusSent = list()

for k in after_trump_titles_only:
    title = k
    n = TextBlob(title)
    ATusSent.append(n.sentiment.polarity)

score = list()
for k in after_trump:
    n = k[3]
    score.append(n)

AT = pd.DataFrame({
    "After Trump News Sentiment": ATusSent,
    "After Trump Score": score
})


In [ ]:
BTusSent=list()
for k in before_trump_titles_only:
    title = k
    n = TextBlob(title)
    BTusSent.append(n.sentiment.polarity)

BTscore = list()
for k in before_trump:
    n = k[3]
    BTscore.append(n)

BT = pd.DataFrame({
    "Before Trump US News Sentiment": BTusSent,
    "After Trump Reddit Score": BTscore
})


In [ ]:
ATusSent_world = list()

for k in after_trump_titles_only_world:
    title = k
    n = TextBlob(title)
    ATusSent_world.append(n.sentiment.polarity)

ATworldscore = list()
for k in after_trump_world:
    n = k[3]
    ATworldscore.append(n)

AT_world = pd.DataFrame({
    "After Trump News Sentiment": ATusSent_world,
    "After Trump Score": ATworldscore
})


In [ ]:
BTusSent_world=list()
for k in before_trump_titles_only_world:
    title = k
    n = TextBlob(title)
    BTusSent_world.append(n.sentiment.polarity)

BTscore_world = list()
for k in before_trump_world:
    n = k[3]
    BTscore_world.append(n)

BT_world = pd.DataFrame({
    "Before Trump World News Sentiment": BTusSent_world,
    "After Trump Reddit Score": BTscore_world
})


In [ ]:
# Before and After Trump Descriptive Statistics on Sentiment for US News
print(AT.describe())
print(BT.describe())

In [ ]:
# Before and After Trump Descriptive Statistics on Sentiment for World News
print(AT_world.describe())
print(BT_world.describe())

In [ ]:
afterTrumpLSA = LSA(after_trump_titles_only)
print(afterTrumpLSA)

In [ ]:
beforeTrumpLSA = LSA(before_trump_titles_only)
print(beforeTrumpLSA)

In [ ]:
beforeTrumpLSA_world = LSA(before_trump_titles_only_world)
print(beforeTrumpLSA_world)

In [ ]:
afterTrumpLSA_world = LSA(after_trump_titles_only_world)
print(afterTrumpLSA_world)

In [ ]:
# Step 4: Visualization - Barplots and Word Clouds
#####barplot########

def getTopN(d, n):
    temp = dict(Counter(d).most_common(n))
    keylist = list()
    freqlist = list()
    for k in temp:
        keylist.append(str.upper(k))
        freqlist.append(temp[k])
    return keylist, freqlist


def barplot(names, freqs, title):
    x_pos = np.arange(len(names))
    plt.figure(figsize=(20, 10))
    plt.bar(x_pos, freqs, color='#FFC222', align='center', alpha=0.5, width=.5)
    plt.xticks(x_pos, names)
    plt.ylabel('Frequency')
    plt.title(title)
    plt.show()

In [ ]:
usNames, usFreq = getTopN(usLSA,15)
barplot(usNames,usFreq,"US Top 15 Title Frequencies")

In [ ]:
worldNames, worldFreq = getTopN(worldLSA,15)
barplot(worldNames,worldFreq,"World Top 15 Title Frequencies")

In [ ]:
beforeTrumpNames, beforeTrumpFreq = getTopN(beforeTrumpLSA,15)
barplot(beforeTrumpNames,beforeTrumpFreq,"Before Trump (US) Top 15 Title Frequencies")

In [ ]:
afterTrumpNames, afterTrumpFreq = getTopN(afterTrumpLSA,15)
barplot(afterTrumpNames,afterTrumpFreq,"After Trump (US) Top 15 Title Frequencies")

In [ ]:
beforeTrumpNamesWorld, beforeTrumpFreqWorld = getTopN(beforeTrumpLSA_world,15)
barplot(beforeTrumpNamesWorld,beforeTrumpFreqWorld,"Before Trump (World) Top 15 Title Frequencies")

In [ ]:
afterTrumpNamesWorld, afterTrumpFreqWorld = getTopN(afterTrumpLSA_world,15)
barplot(afterTrumpNamesWorld,afterTrumpFreqWorld,"After Trump (World) Top 15 Title Frequencies")

In [ ]:
# build stopwords
stopwords = set(STOPWORDS)
#if we need to add custom stop words we can add them here after wordcloud appears
#so we can add any stop words that we may see have too high of frequencies here. code below:
#stopwords.add("said")
stopwords.add("said")
stopwords.add("says")
stopwords.add("say")


%matplotlib inline
#########wordcloud#########
# input a txt file of our words here

# takes list of strings
# outputs wordclouds
def buildWordcloud(t):
    samp = t
    text = " ".join(samp)
    # use the reddit picture i send you here, only png works i think
    reddit_alien = np.array(Image.open("redditpic.png"))
    #change the max words and the font size to see what looks best
    wc = WordCloud(width=1200, height=1100, background_color="white", max_words=500, mask=reddit_alien,
                   stopwords=stopwords, max_font_size=130, random_state=1000)
    # generate word cloud
    wc.generate(text)
    # create coloring from image
    image_colors = ImageColorGenerator(reddit_alien)
    # show
    #makes the picture bigger
    plt.figure( figsize=(20,10) )
    #gives the wordcloud the same colors as pictures
    plt.imshow(wc.recolor(color_func=image_colors), interpolation="bilinear")
    plt.axis("off")
    plt.show()


buildWordcloud(usNews_titles)
buildWordcloud(worldNews_titles)
#
buildWordcloud(before_trump_titles_only)
buildWordcloud(after_trump_titles_only)

In [ ]:
#from our initial barplots we can see that words: say, said, and says are not included in our
#stopwords so we are going to have to manually input these in order to get a better visualiztion
#and a more accurate analysis.

In [ ]:
#makes our list of words all capitals so it looks better in word cloud
us_upper = [x.upper() for x in usNews_titles]
world_upper= [x.upper() for x in worldNews_titles]
bt_upper= [x.upper() for x in before_trump_titles_only]
at_upper= [x.upper() for x in after_trump_titles_only]

In [ ]:
buildWordcloud(us_upper)
buildWordcloud(world_upper)
buildWordcloud(bt_upper)
buildWordcloud(at_upper)

In [ ]:
# Step 5 (Bonus): Comment stuff AND/OR title classifications unsupervised

In [ ]:
#kmeans setup

In [ ]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

# parse commandline arguments
op = OptionParser()
op.add_option("--lsa",
              dest="n_components", type="int",
              help="Preprocess documents with latent semantic analysis.")
op.add_option("--no-minibatch",
              action="store_false", dest="minibatch", default=True,
              help="Use ordinary k-means algorithm (in batch mode).")
op.add_option("--no-idf",
              action="store_false", dest="use_idf", default=True,
              help="Disable Inverse Document Frequency feature weighting.")
op.add_option("--use-hashing",
              action="store_true", default=False,
              help="Use a hashing feature vectorizer")
op.add_option("--n-features", type=int, default=10000,
              help="Maximum number of features (dimensions)"
                   " to extract from text.")
op.add_option("--verbose",
              action="store_true", dest="verbose", default=False,
              help="Print progress reports inside k-means algorithm.")

print(__doc__)
op.print_help()

(opts, args) = op.parse_args([""])

In [ ]:
def kmeans(data_set, k, data_name):
    print()
    print("Kmeans for k = " + str(k) + " on " + data_name)
    print()
    categories = None
    dataset = data_set
    true_k = k

    if opts.use_hashing:
        if opts.use_idf:
            # Perform an IDF normalization on the output of HashingVectorizer
            hasher = HashingVectorizer(n_features=opts.n_features,
                                       stop_words=stopwords, non_negative=True,
                                       norm=None, binary=False)
            vectorizer = make_pipeline(hasher, TfidfTransformer())
        else:
            vectorizer = HashingVectorizer(n_features=opts.n_features,
                                           stop_words=stopwords,
                                           non_negative=False, norm='l2',
                                           binary=False)
    else:
        vectorizer = TfidfVectorizer(max_df=0.5, max_features=opts.n_features,
                                     min_df=2, stop_words=stopwords,
                                     use_idf=opts.use_idf)
    X = vectorizer.fit_transform(dataset)


    if opts.n_components:
        print("Performing dimensionality reduction using LSA")
        t0 = time()
        # Vectorizer results are normalized, which makes KMeans behave as
        # spherical k-means for better results. Since LSA/SVD results are
        # not normalized, we have to redo the normalization.
        svd = TruncatedSVD(opts.n_components)
        normalizer = Normalizer(copy=False)
        lsa = make_pipeline(svd, normalizer)

        X = lsa.fit_transform(X)

        explained_variance = svd.explained_variance_ratio_.sum()
        print("Explained variance of the SVD step: {}%".format(
            int(explained_variance * 100)))

        print()

    if opts.minibatch:
        km = MiniBatchKMeans(n_clusters=true_k, init='k-means++', n_init=1,
                             init_size=1000, batch_size=1000, verbose=opts.verbose)
    else:
        km = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1,
                    verbose=opts.verbose)

    print("Clustering sparse data with %s" % km)
    km.fit(X)
    if not opts.use_hashing:
        print("Top terms per cluster:")

        if opts.n_components:
            original_space_centroids = svd.inverse_transform(km.cluster_centers_)
            order_centroids = original_space_centroids.argsort()[:, ::-1]
        else:
            order_centroids = km.cluster_centers_.argsort()[:, ::-1]

        terms = vectorizer.get_feature_names()
        for i in range(true_k):
            print("Cluster %d:" % i, end='')
            for ind in order_centroids[i, :10]:
                print(' %s' % terms[ind], end='')
            print()

In [ ]:
kmeans(usNews_titles,10, "US News Titles")
kmeans(worldNews_titles,10, "World News Titles")

kmeans(usNews_titles,5, "US News Titles")
kmeans(worldNews_titles,5, "World News Titles")

In [ ]:
#make list of titles into a string and filter out stopwords
from nltk.corpus import stopwords
import numpy as np
import sklearn.cluster
import distance

#takes news list, n size of sample, d dampening (.5 < d < 1)
def affinity_propagation(news_list,n,d):
    np.random.seed(1)
    News_string = ' '.join(news_list)
    #sampleUS = usNews_string[0:2000]
    #sampleUS[0:200]
    News_string_filtered = ' '.join([word for word in News_string.split() if word not in (stopwords.words('english') or custom_stopwords)])
    #with sample of 10,000 is fine. doesnt take too long yet
    sample = News_string_filtered[0:n]
    #below words = to what we want clustered. so need to use this for world too. trump?
    words = sample.split(" ") #Replace this line
    words = np.asarray(words) #So that indexing with a list will work
    lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

    affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=d)
    affprop.fit(lev_similarity)
    for cluster_id in np.unique(affprop.labels_):
        exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
        cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
        cluster_str = ", ".join(cluster)
        print("[%s]: %s" % (str.upper(exemplar), cluster_str))

In [ ]:
affinity_propagation(usNews_titles, 1000, .98)